# IMPORTS

In [2]:
import pandas as pd
import numpy as np
import pickle
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import joblib

KeyboardInterrupt: 

# FLASK

In [7]:
encoding = pd.read_csv('encoding.csv')

In [2]:
sc = StandardScaler()

In [4]:
sc = joblib.load('std_scaler.bin')

In [5]:
# LOADING PRETRAINED MODEL
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.f1 = nn.Linear(20,128)
        self.f2 = nn.Linear(128,64)
        self.f3 = nn.Linear(64,32)
        self.f4 = nn.Linear(32,1)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()
    def forward(self, input1):
        x = self.f1(input1)
        x = self.relu(x)
        x = self.f2(x)
        x = self.relu(x)
        x = self.f3(x)
        x = self.relu(x)
        x = self.f4(x)
        x = self.sig(x)
        return x

In [1]:
app = Flask(__name__)
run_with_ngrok(app)

model = Classifier()
model.load_state_dict(torch.load('saved_weights.pt'))

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    features = [[]]

    if request.form['Gender'] == "Male":
        features[0].append(1)
    else:
        features[0].append(0)
    
    features[0].append(float(encoding[
        (encoding['COLUMNS']=='gender_filled_new_cat_churn_mean')
        & (encoding['COLUMNS_VALUES']==request.form['Gender'])]['VALUES'].values[0]))    

    if request.form['Occupation'] == "salaried":
        features[0].append(1)
    else:
        features[0].append(0)
    
    features[0].append(int(request.form['Dependents']))
    
    features[0].append(float(encoding[
        (encoding['COLUMNS']=='customer_nw_category_cat_churn_sum')
        & (encoding['COLUMNS_VALUES']==request.form['customer_nw_category'])]['VALUES'].values[0]))
    
    features[0].append(float(encoding[
        (encoding['COLUMNS']=='customer_nw_category_cat_churn_mean')
        & (encoding['COLUMNS_VALUES']==request.form['customer_nw_category'])]['VALUES'].values[0]))
    
    features[0].append(int(request.form['customer_nw_category']))
    
    features[0].append(int(request.form['days_since_last_transaction_filled_999']))
    
    features[0].append(np.log(float(request.form['previous_month_credit'])+40))
    
    features[0].append(int(request.form['age']))
    
    features[0].append(np.log(float(request.form['current_month_credit'])+40))
    
    features[0].append(int(request.form['vintage']))
    
    features[0].append(float(request.form['previous_month_end_balance']))
    
    features[0].append(np.log(float(request.form['previous_month_balance'])+40))
    
    features[0].append(np.log(float(request.form['average_monthly_balance_prevQ2'])+40))
    
    features[0].append(np.log(float(request.form['previous_month_debit'])+40))
    
    features[0].append(np.log(float(request.form['average_monthly_balance_prevQ'])+40))
    
    features[0].append(np.log(float(request.form['current_month_balance'])+40))
    
    features[0].append(np.log(float(request.form['current_month_debit'])+40))
    
    features[0].append(np.log(float(request.form['current_balance'])+40))
    
    features = sc.transform(features)
    
    features = torch.tensor(features, dtype=torch.float)
    # PREDICTION FROM SAVED MODEL
    result = model(features)
    if int(result.detach().cpu().squeeze().numpy()) == 1:
        result = "Customer Will Churn"
    else:
        result = "Customer Will Not Churn"
    # SHOWING RESULT TO USER
    return render_template(
        "index.html",
        prediction_text="{}".format(result),
    )

if __name__ == "__main__":
    app.run()

NameError: name 'Flask' is not defined